# Microsoft SEAL : Homomorphic Encryption (HE) in Machine Learning

In this example, we serve inferencing (prediction) for a model trained by regression with Scikit-Learn.<br>
The customer will encrypt data (inputs) with homomorphic encryption (HE) library, Microsoft SEAL. The service provider cannot then know the customer's privacy information, but it can perform operation with this data.

To set up the required software, see the [previous exercise](./03-seal-python-with-pyeva-compiler.ipynb). (Here I use Microsoft SEAL and EVA compiler in Python.)

*back to [Readme](https://github.com/tsmatz/homomorphic-encryption-microsoft-seal/)*

## Install additional packages

In [ ]:
!pip3 install scikit-learn

## Sampling data

First, I create sample dataset for machine learning.<br>
In this example, I create the following data with noise.

$ y = 5.1 x_0 + 2.1 x_1 + 1.5 + \epsilon $

where $ \epsilon $ is Gaussian noise.

In [1]:
import numpy as np

np.random.seed(1000) # For debugging and reproducibility

N = 25 * 25

x0,x1 = np.mgrid[5.0:5.0 + 0.5 * 25:0.5, 5.0:5.0 + 0.5 * 25:0.5]
X = np.vstack((x0.flatten(), x1.flatten())).T

y = x0.flatten() * 5.1 + x1.flatten() * 2.1 + 1.5 + np.random.normal(0.0, 2.0, size=N)

## Generate model

Now I train a model using previous sample dataset. (See [here](https://tsmatz.wordpress.com/2017/08/30/regression-in-machine-learning-math-for-beginners/) for linear regression.)

In [2]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)

The trained model will have the following parameters (coefficients).

In [3]:
reg.coef_

array([5.14201748, 2.07951243])

In [4]:
reg.intercept_

1.271811188192629

## Generate predictor for encryption

With Microsoft SEAL, now I generate the compiled polynomial $ y = b_0 x_0 + b_1 x_1 + b_2 $ for prediction, where $ b_0, b_1, b_2 $ are trained parameters.

In [5]:
# Define program
from eva import *
poly = EvaProgram("Polynomial", vec_size=1024)
with poly:
    x0 = Input("x0")
    x1 = Input("x1")
    Output("y", reg.coef_[0]*x0 + reg.coef_[1]*x1 + reg.intercept_)
poly.set_output_ranges(30)
poly.set_input_scales(30)
# Compile program with CKKS scheme
from eva.ckks import *
compiler = CKKSCompiler()
compiled_poly, params, signature = compiler.compile(poly)

The ```params``` object (```CKKSParameters```) holds polyModulusDegree, primeBits (list of number of bits each prime should have), and rotation (list of steps for generating galois key).<br>
You can save and serialize this ```params``` for sharing.

> Note : See [here](./02-seal-python-ckks-with-c-wrapper.ipynb) for details about polyModulusDegree and primeBits.

In [6]:
from eva import save, load
save(params, 'poly.evaparams')

## Generate keys

Here I create public key, secret key, relin key, and galois key.<br>
The generated public context (which holds public key, relin key, and galois key) can be serialized and shared as follows.<br>
(On contrary, the secret context can be hold on only customer and should not be shared.)

In [7]:
# Generate key context
from eva.seal import *
params = load('poly.evaparams')
public_ctx, secret_ctx = generate_keys(params)
# Save public context
save(public_ctx, 'poly.sealpublic')

## Generate encrypted input

Now the customer (client) will generate encrypted inputs with public key.<br>
Here I set 3 inputs, $ (x_0, x_1) = (1.1, 1.1), (2.2, 2.2), (3.3, 3.3) $.

In [8]:
# Create encryption for x0 and x1
inputs = { "x0": [0.0 for i in range(compiled_poly.vec_size)], "x1": [0.0 for i in range(compiled_poly.vec_size)] }
inputs["x0"][0] = 1.1
inputs["x0"][1] = 2.2
inputs["x0"][2] = 3.3
inputs["x1"][0] = 1.1
inputs["x1"][1] = 2.2
inputs["x1"][2] = 3.3
public_ctx = load('poly.sealpublic')
encInputs = public_ctx.encrypt(inputs, signature)
# Save (serialize) encrypted byte to file
from eva import save
save(encInputs, 'poly_inputs.sealvals')

## Execute computation

On service side, execute computation with shared public context (which include relin key).

In [9]:
# Compute with homomorphic encryption (HE)
public_ctx = load('poly.sealpublic')
encOutputs = public_ctx.execute(compiled_poly, encInputs)
# Serialize results
save(encOutputs, 'poly_outputs.sealvals')

## Decrypt results

The customer can decrypt results with his secret key. (Only this customer can see the inputs and results.)

In [10]:
encOutputs = load('poly_outputs.sealvals')
outputs = secret_ctx.decrypt(encOutputs, signature)
print("********** Result is **********")
for i in range(3):
    print(outputs["y"][i])
# Actual result is 5.1*x0 + 2.1*x1 + 0.5

********** Result is **********
9.21549823384632
17.159159768683594
25.102852135244238
